# IR Caplets Volatility Parametrization

In this Notebook, we explore three parametrization of IR caplets volatility surface by varying the `X_axis` and `Y_axis` inputs,  `IndexName` & `IndexTenor` fields. This process helps define the request specification for **IR Caplets Volatility Surfaces** within the **LSEG Financial Analytics SDK**. 

We will analyze the following combinations:

1. **X_axis:** `STRIKE`, **Y_axis:** `TENOR`
2. **X_axis:** `STRIKE`, **Y_axis:** `DATE`
3. **X_axis:** `MONEYNESS`,  **Y_axis:** `TENOR`
4. Change caplets `IndexName` & `IndexTenor` USD-SOFR-ON to USD-LIBOR-3M



## Imports

Import the following necessary modules:

- `ircaplet_volatility` - for Ir Caplet Volatility surface construction

This notebook uses external libraries **pandas, numpy, Ipython, matplotlib**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.



In [1]:
from lseg_analytics.pricing.market_data import ircaplet_volatility as cv

import pandas as pd
import numpy as np
import json
import datetime as dt
import matplotlib.pyplot as plt
from IPython.display import display


## 1. Caplets Volatility Surface - Strike, Tenor

 We start from a volatility surface with X_axis: `STRIKE`and Y_axis: `TENOR`

### Data Preparation

In [2]:
print("Step 1: Creating Surface Definition...")

currency = "USD"
index_name = "SOFR"

# Create surface definition object
surface_definition = cv.CapletsStrippingDefinition(
        instrument_code = currency,
        index_name = index_name,
        reference_caplet_tenor = "ON"
        )
print(f"   ✓ Instrument: {surface_definition.instrument_code}")

print("Step 2: Configuring Surface Parameters...")
surface_parameters = cv.CapletsStrippingSurfaceParameters(
        calculation_date = dt.datetime.strptime("2025-01-18", "%Y-%m-%d"),
        x_axis = cv.XAxisEnum.STRIKE,                                    # Options: DATE, DELTA, EXPIRY, MONEYNESS, STRIKE, TENOR
        y_axis = cv.YAxisEnum.TENOR                                      # Options: same as X-axis
    )
print(f"   ✓ Surface Parameters: {surface_parameters}")

print("Step 3: Create request item...")
# Create the main request object with basic configuration
request_item = cv.CapletsStrippingSurfaceRequestItem(
        surface_tag = f"{currency}_CAPLET_VOLSURFACE",
        underlying_definition = surface_definition,
        surface_parameters = surface_parameters,
        underlying_type = cv.CurvesAndSurfacesUnderlyingTypeEnum.Cap,
        surface_layout = cv.SurfaceOutput(
            format = cv.FormatEnum.Matrix,  # Options: List, Matrix 
        )
    )
print(f"   ✓ Request Item: {json.dumps(request_item.as_dict(), indent=4)}")

Step 1: Creating Surface Definition...
   ✓ Instrument: USD
Step 2: Configuring Surface Parameters...
   ✓ Surface Parameters: {'calculationDate': '2025-01-18T00:00:00Z', 'xAxis': 'Strike', 'yAxis': 'Tenor'}
Step 3: Create request item...
   ✓ Request Item: {
    "surfaceTag": "USD_CAPLET_VOLSURFACE",
    "underlyingDefinition": {
        "instrumentCode": "USD",
        "indexName": "SOFR",
        "referenceCapletTenor": "ON"
    },
    "surfaceParameters": {
        "calculationDate": "2025-01-18T00:00:00Z",
        "xAxis": "Strike",
        "yAxis": "Tenor"
    },
    "underlyingType": "Cap",
    "surfaceLayout": {
        "format": "Matrix"
    }
}


### Request Execution

In [3]:
# Execute the calculation using the calculate function
try:
    response = cv.calculate(universe=[request_item])

    # Display response structure information
    surface_data = response['data'][0]
    if 'surface' in surface_data:
        print(f"   Calculation successful!")
        print(f"   Surface data points available: {len(surface_data['surface']) - 1} x {len(surface_data['surface'][0]) - 1}")
    else:
        print("   No surface data found in response")
    
except Exception as e:
    print(f"   Calculation failed: {str(e)}")
    raise

   Calculation successful!
   Surface data points available: 14 x 25


### Results Display 

In [4]:
# Access surface matrix data from the response
surface_data = response['data'][0]['surface']

# Extract strikes (column headers) and tenors (row headers)
strikes = surface_data[0][1:]  # First row, excluding first element
tenors = [row[0] for row in surface_data[1:]]  # First column, excluding header row
volatility_matrix = np.array([[float(val) for val in row[1:]] for row in surface_data[1:]])

# Create DataFrame for easier manipulation and display
surface_df = pd.DataFrame(volatility_matrix, index=tenors, columns=strikes)

# Extract axis names for labeling plots
x_axis = surface_parameters.x_axis.name
y_axis = surface_parameters.y_axis.name

print("Surface DataFrame Info:") 
print(f"   Shape: {surface_df.shape} (rows × columns)") 
print(f"   x_axis: {x_axis}") 
print(f"   y_axis: {y_axis}") 

Surface DataFrame Info:
   Shape: (14, 25) (rows × columns)
   x_axis: STRIKE
   y_axis: TENOR


#### Caplets Normal Volatility Matrix (bp) [Strike, Tenor]

In [5]:
# Display all columns using context manager (temporary setting)
with pd.option_context('display.max_columns', None, 'display.width', None):
    display(surface_df)

,0.250000,0.500000,0.750000,1.000000,1.500000,2.000000,2.500000,3.000000,4.000000,4.009199,4.009324,4.010736,4.012121,4.012322,4.015504,4.019531,4.027613,4.028712,4.032534,4.045894,4.066390,4.112150,5.000000,6.000000,7.000000
3D,167.90,163.20,158.10,152.70,141.30,128.80,115.40,100.70,66.70,66.28,66.28,66.21,66.15,66.14,65.99,65.81,65.44,65.39,65.22,64.61,63.68,61.60,73.50,99.70,124.60
9M,167.90,163.20,158.10,152.70,141.30,128.80,115.40,100.70,66.70,66.28,66.28,66.21,66.15,66.14,65.99,65.81,65.44,65.39,65.22,64.61,63.68,61.60,73.50,99.70,124.60
1Y9M,129.84,128.59,127.14,125.80,122.90,120.46,118.77,118.43,126.21,126.03,126.03,126.00,125.97,125.97,125.91,125.83,125.67,125.65,125.64,125.61,125.56,125.45,123.35,125.76,137.49
2Y9M,128.29,126.77,125.17,123.18,119.19,115.33,111.42,108.28,103.14,102.61,102.60,102.52,102.44,102.44,102.48,102.53,102.64,102.65,102.70,102.87,103.13,103.71,115.00,131.57,146.63
3Y9M,115.66,114.75,113.92,113.06,111.40,110.05,109.17,109.54,115.14,115.73,115.74,115.75,115.75,115.75,115.76,115.78,115.81,115.81,115.83,115.87,115.95,116.12,119.39,126.85,137.92
4Y9M,110.54,109.81,108.49,107.74,106.09,104.38,103.52,103.32,104.40,104.34,104.34,104.35,104.36,104.36,104.39,104.43,104.50,104.51,104.54,104.66,104.84,105.24,113.00,123.93,135.39
5Y9M,103.72,103.11,102.55,101.68,100.35,99.71,99.57,99.93,104.90,104.96,104.96,104.97,104.97,104.98,104.99,105.01,105.05,105.05,105.07,105.13,105.23,105.45,109.66,118.33,128.49
6Y9M,102.51,101.79,101.34,100.49,99.72,98.07,98.00,98.45,100.28,100.89,100.90,101.00,101.09,101.10,101.13,101.16,101.22,101.23,101.26,101.37,101.53,101.89,108.88,117.77,128.09
7Y9M,97.91,97.52,96.82,96.22,94.65,95.20,94.78,95.67,100.55,100.28,100.28,100.24,100.20,100.19,100.10,100.13,100.17,100.18,100.20,100.28,100.39,100.66,105.76,114.21,124.19
8Y9M,96.40,95.30,95.02,94.55,94.04,92.58,93.21,92.89,96.23,96.89,96.90,97.00,97.10,97.11,97.34,97.63,97.69,97.70,97.72,97.81,97.95,98.25,104.18,112.39,122.21


## 2. Caplets Volatility Surface - Strike, Date

### Data Preparation
 Now we will keep X_axis:`STRIKE`and change Y_axis: from `TENOR` to `DATE`

In [6]:
# Keep X axis to use STRIKE
surface_parameters.x_axis = cv.XAxisEnum.STRIKE
print(f"   ✓ X-axis changed to: {surface_parameters.x_axis.name}")

# Change Y axis to use DATE
surface_parameters.y_axis = cv.XAxisEnum.DATE
print(f"   ✓ Y-axis changed to: {surface_parameters.y_axis.name}")

   ✓ X-axis changed to: STRIKE
   ✓ Y-axis changed to: DATE


### Request Execution

In [7]:
# Execute the calculation using the calculate function
try:
    response = cv.calculate(universe=[request_item])

    # Display response structure information
    surface_data = response['data'][0]
    if 'surface' in surface_data:
        print(f"   Calculation successful!")
        print(f"   Surface data points available: {len(surface_data['surface']) - 1} x {len(surface_data['surface'][0]) - 1}")
    else:
        print("   No surface data found in response")
    
except Exception as e:
    print(f"   Calculation failed: {str(e)}")
    raise

   Calculation successful!
   Surface data points available: 14 x 25


### Results Display

In [8]:
# Access surface matrix data from the response
surface_data = response['data'][0]['surface']

# Extract strikes (column headers) and tenors (row headers)
strikes = surface_data[0][1:]  # First row, excluding first element
tenors = [row[0] for row in surface_data[1:]]  # First column, excluding header row
volatility_matrix = np.array([[float(val) for val in row[1:]] for row in surface_data[1:]])

# Create DataFrame for easier manipulation and display
surface_df = pd.DataFrame(volatility_matrix, index=tenors, columns=strikes)

# Extract axis names for labeling plots
x_axis = surface_parameters.x_axis.name
y_axis = surface_parameters.y_axis.name

print("Surface DataFrame Info:") 
print(f"   Shape: {surface_df.shape} (rows × columns)") 
print(f"   x_axis: {x_axis}") 
print(f"   y_axis: {y_axis}") 


Surface DataFrame Info:
   Shape: (14, 25) (rows × columns)
   x_axis: STRIKE
   y_axis: DATE


#### Caplets Normal Volatility Matrix (bp) [Strike, Date]

In [9]:
# Display all columns using context manager (temporary setting)
with pd.option_context('display.max_columns', None, 'display.width', None):
    display(surface_df)

,0.250000,0.500000,0.750000,1.000000,1.500000,2.000000,2.500000,3.000000,4.000000,4.009199,4.009324,4.010736,4.012121,4.012322,4.015504,4.019531,4.027613,4.028712,4.032534,4.045894,4.066390,4.112150,5.000000,6.000000,7.000000
2025-01-21,167.90,163.20,158.10,152.70,141.30,128.80,115.40,100.70,66.70,66.28,66.28,66.21,66.15,66.14,65.99,65.81,65.44,65.39,65.22,64.61,63.68,61.60,73.50,99.70,124.60
2025-10-21,167.90,163.20,158.10,152.70,141.30,128.80,115.40,100.70,66.70,66.28,66.28,66.21,66.15,66.14,65.99,65.81,65.44,65.39,65.22,64.61,63.68,61.60,73.50,99.70,124.60
2026-10-21,129.84,128.59,127.14,125.80,122.90,120.46,118.77,118.43,126.21,126.03,126.03,126.00,125.97,125.97,125.91,125.83,125.67,125.65,125.64,125.61,125.56,125.45,123.35,125.76,137.49
2027-10-21,128.29,126.77,125.17,123.18,119.19,115.33,111.42,108.28,103.14,102.61,102.60,102.52,102.44,102.44,102.48,102.53,102.64,102.65,102.70,102.87,103.13,103.71,115.00,131.57,146.63
2028-10-23,115.66,114.75,113.92,113.06,111.40,110.05,109.17,109.54,115.14,115.73,115.74,115.75,115.75,115.75,115.76,115.78,115.81,115.81,115.83,115.87,115.95,116.12,119.39,126.85,137.92
2029-10-22,110.54,109.81,108.49,107.74,106.09,104.38,103.52,103.32,104.40,104.34,104.34,104.35,104.36,104.36,104.39,104.43,104.50,104.51,104.54,104.66,104.84,105.24,113.00,123.93,135.39
2030-10-21,103.72,103.11,102.55,101.68,100.35,99.71,99.57,99.93,104.90,104.96,104.96,104.97,104.97,104.98,104.99,105.01,105.05,105.05,105.07,105.13,105.23,105.45,109.66,118.33,128.49
2031-10-21,102.51,101.79,101.34,100.49,99.72,98.07,98.00,98.45,100.28,100.89,100.90,101.00,101.09,101.10,101.13,101.16,101.22,101.23,101.26,101.37,101.53,101.89,108.88,117.77,128.09
2032-10-21,97.91,97.52,96.82,96.22,94.65,95.20,94.78,95.67,100.55,100.28,100.28,100.24,100.20,100.19,100.10,100.13,100.17,100.18,100.20,100.28,100.39,100.66,105.76,114.21,124.19
2033-10-24,96.40,95.30,95.02,94.55,94.04,92.58,93.21,92.89,96.23,96.89,96.90,97.00,97.10,97.11,97.34,97.63,97.69,97.70,97.72,97.81,97.95,98.25,104.18,112.39,122.21


## 3. Caplets Volatility Surface - Moneyness, Tenor

### Data Preparation
 Here we will change X_axis from `STRIKE` to `MONEYNESS`and Y_axis: from `DATE` to `TENOR`

In [10]:
# Change X axis to use MONEYNESS
surface_parameters.x_axis = cv.XAxisEnum.MONEYNESS
print(f"   ✓ X-axis changed to: {surface_parameters.x_axis.name}")

# Change Y axis to use TENOR
surface_parameters.y_axis = cv.XAxisEnum.TENOR
print(f"   ✓ Y-axis changed to: {surface_parameters.y_axis.name}")

   ✓ X-axis changed to: MONEYNESS
   ✓ Y-axis changed to: TENOR


### Request Execution

In [11]:
# Execute the calculation using the calculate function
try:
    response = cv.calculate(universe=[request_item])

    # Display response structure information
    surface_data = response['data'][0]
    if 'surface' in surface_data:
        print(f"   Calculation successful!")
        print(f"   Surface data points available: {len(surface_data['surface']) - 1} x {len(surface_data['surface'][0]) - 1}")
    else:
        print("   No surface data found in response")
    
except Exception as e:
    print(f"   Calculation failed: {str(e)}")
    raise

   Calculation successful!
   Surface data points available: 14 x 26


### Results Display

In [12]:
# Access surface matrix data from the response
surface_data = response['data'][0]['surface']

# Extract strikes (column headers) and tenors (row headers)
strikes = surface_data[0][1:]  # First row, excluding first element
tenors = [row[0] for row in surface_data[1:]]  # First column, excluding header row
volatility_matrix = np.array([[float(val) for val in row[1:]] for row in surface_data[1:]])

# Create DataFrame for easier manipulation and display
surface_df = pd.DataFrame(volatility_matrix, index=tenors, columns=strikes)

# Extract axis names for labeling plots
x_axis = surface_parameters.x_axis.name
y_axis = surface_parameters.y_axis.name

print("Surface DataFrame Info:") 
print(f"   Shape: {surface_df.shape} (rows × columns)") 
print(f"   x_axis: {x_axis}") 
print(f"   y_axis: {y_axis}") 


Surface DataFrame Info:
   Shape: (14, 26) (rows × columns)
   x_axis: MONEYNESS
   y_axis: TENOR


#### Caplets Normal Volatility Matrix (bp) [Moneyness, Tenor]

In [13]:
# Display all columns using context manager (temporary setting)
with pd.option_context('display.max_columns', None, 'display.width', None):
    display(surface_df)

,-90.000000,-85.000000,-80.000000,-75.000000,-70.000000,-65.000000,-60.000000,-55.000000,-50.000000,-45.000000,-40.000000,-35.000000,-30.000000,-25.000000,-20.000000,-15.000000,-10.000000,-5.000000,0.000000,10.000000,20.000000,30.000000,40.000000,50.000000,60.000000,70.000000
3D,164.90,160.87,156.56,152.08,147.49,142.74,137.78,132.63,127.35,121.95,116.32,110.39,104.29,98.24,92.20,85.85,78.83,70.79,61.60,58.22,71.16,84.11,94.16,103.66,113.82,124.36
9M,164.90,160.87,156.56,152.08,147.49,142.74,137.78,132.63,127.35,121.95,116.32,110.39,104.29,98.24,92.20,85.85,78.83,70.79,61.60,58.22,71.16,84.11,94.16,103.66,113.82,124.36
1Y9M,129.09,127.98,126.82,125.74,124.58,123.39,122.24,121.22,120.38,119.71,119.05,118.24,117.80,118.71,121.39,124.72,127.37,127.99,125.62,124.52,123.57,123.21,123.93,126.28,130.46,135.75
2Y9M,127.37,126.15,124.78,123.15,121.53,119.95,118.36,116.79,115.28,113.85,112.27,110.32,108.58,108.32,110.20,112.63,113.53,110.81,102.45,107.42,112.44,118.32,125.07,131.89,138.18,144.07
3Y9M,115.10,114.41,113.75,113.05,112.35,111.69,111.11,110.60,110.04,109.42,109.09,109.47,110.05,109.48,107.06,104.37,103.53,106.62,115.75,117.13,118.55,120.56,123.41,127.00,131.20,135.77
4Y9M,110.18,109.28,108.28,107.74,107.15,106.45,105.69,104.96,104.37,103.98,103.68,103.33,103.07,103.35,104.38,105.65,106.51,106.27,104.33,107.78,111.24,115.12,119.49,124.09,128.69,133.28
5Y9M,103.33,102.90,102.39,101.67,101.04,100.55,100.17,99.89,99.70,99.61,99.57,99.57,99.66,99.95,100.55,101.41,102.48,103.69,104.97,106.76,108.59,111.15,114.58,118.48,122.52,126.62
6Y9M,102.03,101.62,101.19,100.48,100.10,99.88,99.45,98.75,98.06,97.63,97.72,98.51,99.26,98.35,94.95,90.99,89.00,91.51,101.07,104.24,107.39,110.67,114.18,117.95,121.98,126.18
7Y9M,97.71,97.24,96.69,96.20,95.48,94.82,94.63,94.90,95.21,95.20,94.94,94.58,94.60,95.78,98.49,101.66,103.91,103.85,100.12,102.33,104.59,107.34,110.68,114.42,118.36,122.41
8Y9M,95.64,95.14,94.94,94.54,94.35,94.20,93.76,93.09,92.57,92.50,92.90,93.67,94.06,92.72,89.08,85.19,83.60,86.87,97.56,100.28,102.94,105.84,109.08,112.65,116.49,120.50


## 4. Change Caplets `IndexName` & `IndexTenor`

### Data Preparation
 Here we will change both the caplets `IndexName` & `IndexTenor` to `USD-LIBOR-3M`. We have X_axis: `STRIKE`and Y_axis: `TENOR`

In [14]:
print("Step 1: Creating Surface Definition...")

# Update caplets index to LIBOR 3M
currency = "USD"
index_name = "LIBOR" # index name changed from SOFR to LIBOR

# Create surface definition object
surface_definition = cv.CapletsStrippingDefinition(
        instrument_code = currency,
        index_name = index_name,
        reference_caplet_tenor = "3M" # index caplet tenor changed from ON to 3M
        )
print(f"   ✓ Instrument: {surface_definition.instrument_code}")

print("Step 2: Configuring Surface Parameters...")
surface_parameters = cv.CapletsStrippingSurfaceParameters(
        calculation_date = dt.datetime.strptime("2025-01-18", "%Y-%m-%d"),
        x_axis = cv.XAxisEnum.STRIKE,                                    # Options: DATE, DELTA, EXPIRY, MONEYNESS, STRIKE, TENOR
        y_axis = cv.YAxisEnum.TENOR                                      # Options: same as X-axis
    )
print(f"   ✓ Surface Parameters: {surface_parameters}")

print("Step 3: Create request item...")
# Create the main request object with basic configuration
request_item = cv.CapletsStrippingSurfaceRequestItem(
        surface_tag = f"{currency}_CAPLET_VOLSURFACE",
        underlying_definition = surface_definition,
        surface_parameters = surface_parameters,
        underlying_type = cv.CurvesAndSurfacesUnderlyingTypeEnum.Cap,
        surface_layout = cv.SurfaceOutput(
            format = cv.FormatEnum.Matrix,  # Options: List, Matrix 
        )
    )
print(f"   ✓ Request Item: {json.dumps(request_item.as_dict(), indent=4)}")

Step 1: Creating Surface Definition...
   ✓ Instrument: USD
Step 2: Configuring Surface Parameters...
   ✓ Surface Parameters: {'calculationDate': '2025-01-18T00:00:00Z', 'xAxis': 'Strike', 'yAxis': 'Tenor'}
Step 3: Create request item...
   ✓ Request Item: {
    "surfaceTag": "USD_CAPLET_VOLSURFACE",
    "underlyingDefinition": {
        "instrumentCode": "USD",
        "indexName": "LIBOR",
        "referenceCapletTenor": "3M"
    },
    "surfaceParameters": {
        "calculationDate": "2025-01-18T00:00:00Z",
        "xAxis": "Strike",
        "yAxis": "Tenor"
    },
    "underlyingType": "Cap",
    "surfaceLayout": {
        "format": "Matrix"
    }
}


### Request Execution

In [15]:
# Execute the calculation using the calculate function
try:
    response = cv.calculate(universe=[request_item])

    # Display response structure information
    surface_data = response['data'][0]
    if 'surface' in surface_data:
        print(f"   Calculation successful!")
        print(f"   Surface data points available: {len(surface_data['surface']) - 1} x {len(surface_data['surface'][0]) - 1}")
    else:
        print("   No surface data found in response")
    
except Exception as e:
    print(f"   Calculation failed: {str(e)}")
    raise

   Calculation successful!
   Surface data points available: 14 x 25


### Results Display 

In [16]:
# Access surface matrix data from the response
surface_data = response['data'][0]['surface']

# Extract strikes (column headers) and tenors (row headers)
strikes = surface_data[0][1:]  # First row, excluding first element
tenors = [row[0] for row in surface_data[1:]]  # First column, excluding header row
volatility_matrix = np.array([[float(val) for val in row[1:]] for row in surface_data[1:]])

# Create DataFrame for easier manipulation and display
surface_df = pd.DataFrame(volatility_matrix, index=tenors, columns=strikes)

# Extract axis names for labeling plots
x_axis = surface_parameters.x_axis.name
y_axis = surface_parameters.y_axis.name

print("Surface DataFrame Info:") 
print(f"   Shape: {surface_df.shape} (rows × columns)") 
print(f"   x_axis: {x_axis}") 
print(f"   y_axis: {y_axis}") 

Surface DataFrame Info:
   Shape: (14, 25) (rows × columns)
   x_axis: STRIKE
   y_axis: TENOR


#### Caplets Normal Volatility Matrix (bp) [`USD-LIBOR-3M`]

In [17]:
# Display all columns using context manager (temporary setting)
with pd.option_context('display.max_columns', None, 'display.width', None):
    display(surface_df)

,0.250000,0.500000,0.750000,1.000000,1.500000,2.000000,2.500000,3.000000,4.000000,4.274068,4.274990,4.277412,4.279588,4.281811,4.287968,4.292547,4.295519,4.303228,4.310811,4.321608,4.340850,4.346952,5.000000,6.000000,7.000000
3M,175.90,171.70,167.20,162.30,151.60,139.70,126.70,112.40,78.80,67.82,67.78,67.69,67.60,67.51,67.26,67.08,66.96,66.65,66.35,65.92,65.14,64.90,69.40,94.50,120.00
9M,175.90,171.70,167.20,162.30,151.60,139.70,126.70,112.40,78.80,67.82,67.78,67.69,67.60,67.51,67.26,67.08,66.96,66.65,66.35,65.92,65.14,64.90,69.40,94.50,120.00
1Y9M,160.43,158.25,156.11,153.78,149.21,144.84,140.96,137.96,137.47,139.12,139.12,139.14,139.15,139.14,139.09,139.06,139.04,138.98,138.93,138.85,138.71,138.67,133.98,127.32,132.08
2Y9M,129.53,128.14,126.36,124.75,121.00,117.22,113.47,110.13,106.28,105.78,105.78,105.81,105.84,105.87,105.95,106.00,106.04,106.14,106.24,106.37,106.62,106.69,114.97,134.91,153.05
3Y9M,124.39,123.31,122.17,121.12,118.82,116.61,114.98,114.38,115.93,117.96,117.96,117.97,117.97,117.98,117.99,118.00,118.01,118.03,118.04,118.07,118.11,118.12,119.56,123.16,132.20
4Y9M,114.36,113.18,112.56,111.07,109.35,107.45,106.30,105.05,106.85,107.30,107.30,107.30,107.32,107.34,107.39,107.43,107.45,107.51,107.58,107.67,107.83,107.88,113.27,126.06,138.01
5Y9M,108.41,108.27,106.83,106.60,104.98,103.86,102.72,103.27,105.16,107.25,107.26,107.28,107.29,107.31,107.34,107.36,107.37,107.41,107.45,107.50,107.59,107.62,110.68,116.57,126.10
6Y9M,106.99,105.31,105.14,104.30,101.99,100.51,100.10,99.40,101.64,102.73,102.74,102.75,102.75,102.76,102.79,102.81,102.83,102.88,102.92,102.98,103.09,103.13,106.90,116.70,126.53
7Y9M,101.28,101.29,100.38,99.15,98.80,98.22,97.51,97.57,100.79,102.63,102.64,102.65,102.67,102.68,102.72,102.75,102.77,102.82,102.87,102.94,103.06,103.10,107.18,113.88,124.06
8Y9M,94.55,93.92,92.89,92.85,92.50,91.56,91.16,91.93,94.42,95.67,95.67,95.68,95.69,95.70,95.73,95.75,95.77,95.82,95.87,95.94,96.07,96.11,100.55,109.58,119.08
